In [21]:
!pip freeze | grep scikit-learn

scikit-learn @ file:///tmp/build/80754af9/scikit-learn_1642617106979/work
scikit-learn-intelex==2021.20220215.212715


In [22]:
import os

import pickle
import pandas as pd

In [23]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [24]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [25]:
year=2021
month=2

df = read_data(f'https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_{year:4d}-{month:02d}.parquet')

In [26]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [27]:
import numpy as np
A = np.array(y_pred)

In [28]:
np.mean(A)

16.191691679979066

In [31]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [35]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['prediction'] = y_pred

In [36]:
df_result

,ride_id,prediction
1,2021/02_1,14.539865
2,2021/02_2,13.740422
3,2021/02_3,15.593339
4,2021/02_4,15.188118
5,2021/02_5,13.817206
...,...,...
1037687,2021/02_1037687,12.433246
1037688,2021/02_1037688,12.910885
1037689,2021/02_1037689,15.834923
1037690,2021/02_1037690,16.783176


In [39]:
output_file = "output_file.parquet"
df_result.to_parquet(
    output_file,
    engine='pyarrow',   
    compression=None,
    index=False
)

In [48]:

file_size = os.system('ls -lh output_file.parquet')
print(file_size)

-rw-rw-r-- 1 asurace asurace 19M Jun 26 18:56 output_file.parquet
